In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [29]:
outages_df = pd.read_excel('data/original/Book2(1).xlsx', usecols=['Унікод АЗС', 'ДатаЧас', 'Обсяг спожитих, кВт', 'Year', 'Month', 'Date', 'Day', 'DayWeek', 'Hour'])

In [30]:
latitude = []
longitude = []

for i in outages_df['Унікод АЗС']:
    if i == 40810600:
        latitude.append(50.418366003690984)
        longitude.append(29.840546770260875)
    if i == 40861000:
        latitude.append(50.31828636812571)
        longitude.append(31.03586158987561)
    if i == 40860500:
        latitude.append(50.319652674621935)
        longitude.append(31.03398680305577)
    if i == 40810700:
        latitude.append(50.42274230627166)
        longitude.append(29.843808164165303)

outages_df['Latitude'] = latitude
outages_df['Longitude'] = longitude


In [31]:
outages_df

,Унікод АЗС,ДатаЧас,"Обсяг спожитих, кВт",Year,Month,Date,Day,DayWeek,Hour,Latitude,Longitude
0,40810600,2023-06-01 00:00:00,29.3216,2023,6,2023-06-01,1,3,0,50.418366,29.840547
1,40810600,2023-06-01 01:00:00,28.2432,2023,6,2023-06-01,1,3,1,50.418366,29.840547
2,40810600,2023-06-01 02:00:00,25.5184,2023,6,2023-06-01,1,3,2,50.418366,29.840547
3,40810600,2023-06-01 03:00:00,28.1296,2023,6,2023-06-01,1,3,3,50.418366,29.840547
4,40810600,2023-06-01 04:00:00,26.8352,2023,6,2023-06-01,1,3,4,50.418366,29.840547
...,...,...,...,...,...,...,...,...,...,...,...
49921,40810700,2024-11-01 19:00:00,59.8150,2024,11,2024-11-01,1,4,19,50.422742,29.843808
49922,40810700,2024-11-01 20:00:00,69.7740,2024,11,2024-11-01,1,4,20,50.422742,29.843808
49923,40810700,2024-11-01 21:00:00,75.8230,2024,11,2024-11-01,1,4,21,50.422742,29.843808
49924,40810700,2024-11-01 22:00:00,70.4940,2024,11,2024-11-01,1,4,22,50.422742,29.843808


In [32]:
outages_list = []
kW_list = []
for i in outages_df['Обсяг спожитих, кВт']:
    if pd.isna(i):
        outages_list.append(False)
        kW_list.append(0)
    else:
        outages_list.append(True)
        kW_list.append(i)

outages_df['Light On'] = outages_list
outages_df['kW'] = kW_list

In [33]:
outages_df = outages_df.drop(columns = ['Date'])
# outages_df = outages_df.drop(columns = ['ДатаЧас'])
outages_df = outages_df.drop(columns = ['Обсяг спожитих, кВт'])

In [34]:
X = outages_df.drop(columns=['kW'])
y = outages_df[['kW', 'Year', 'Month']] 

In [35]:
X_val = X[(X['Year'] ==2024) & (X['Month'] == 10)]
y_val = y[(y['Year'] ==2024) & (y['Month'] == 10)]

X_test = X[(X['Year'] ==2024) & (X['Month'] == 11)]
y_test = y[(y['Year'] ==2024) & (y['Month'] == 11)]

X_train = X[~((X['Year'] == 2024) & (X['Month'] == 11))]
y_train = y[~((y['Year'] == 2024) & (y['Month'] == 11))]

X_train = X_train[(~((X_train['Year'] == 2024) & (X_train['Month'] == 10))) | (~((y_train['Year'] == 2024) & (y_train['Month'] == 11)))]

In [40]:
X_val = X_val.drop(columns = ['ДатаЧас'])
y_val = y_val.drop(columns = ['Year', 'Month'])

X_test = X_test.drop(columns = ['ДатаЧас'])
y_test = y_test.drop(columns = ['Year', 'Month'])

X_train = X_train.drop(columns = ['ДатаЧас'])
y_train = y_train.drop(columns = ['Year', 'Month'])

KeyError: "['ДатаЧас'] not found in axis"

In [44]:
y_test

,kW
12458,32.9008
12459,32.3248
12460,31.7184
12461,31.2640
12462,33.6432
...,...
49921,59.8150
49922,69.7740
49923,75.8230
49924,70.4940


In [49]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error as mse
from tqdm import tqdm

In [51]:
max_depth_list = [4, 6, 8, 10, 12, 16, 18, 20]
n_estimators_list = [50, 100, 200, 500, 1000]
learning_rate_list = [0.1, 0.05, 0.01]

# Flatten all combinations of parameters into one list
param_combinations = [(max_depth, n_estimators, learning_rate) 
                      for max_depth in max_depth_list
                      for n_estimators in n_estimators_list
                      for learning_rate in learning_rate_list]

best_max_depth = None
best_n_estimators = None
best_learning_rate = None
best_validation_preds = None
best_rmse = float('inf')
best_xgb_model = None

# Use a single progress bar to track the entire grid search
for max_depth, n_estimators, learning_rate in tqdm(param_combinations, desc="Grid search progress"):
    xgb_model = xgb.XGBRegressor(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, n_jobs=4, tree_method='auto')
    xgb_model.fit(X_train, y_train)
    validation_preds = xgb_model.predict(X_val)
    validation_preds = pd.Series(validation_preds)
    validation_target = pd.Series(y_val['kW'].values)
    err = np.sqrt(mse(validation_target, validation_preds))
    
    if err < best_rmse:
        best_rmse = err
        best_max_depth = max_depth
        best_n_estimators = n_estimators
        best_learning_rate = learning_rate
        best_validation_preds = validation_preds
        best_xgb_model = xgb_model

print(f'XGB RMSE: {best_rmse}')
print(f'max_depth: {best_max_depth}, n_estimators: {best_n_estimators}, learning_rate: {best_learning_rate}')

Grid search progress: 100%|██████████| 120/120 [08:39<00:00,  4.33s/it]

XGB RMSE: 0.2411003406127406
max_depth: 20, n_estimators: 1000, learning_rate: 0.1


In [38]:
print(X_train.shape, y_train.shape)

(49830, 9) (49830, 1)
